## spark read s3 delta

In [42]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window, Row

# from PyQt5.QtWidgets import QApplication
# from PyQt5.QtWebEngineWidgets import QWebEngineView
import pandas as pd
from pyspark.sql import SparkSession
# from iPython import display

import pprint
import boto3
import json
import sys
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

imported modules.


In [14]:
client = boto3.client('s3')
bucket = "yelp-dataset-stevenhurwitt"
file = "raw/yelp_academic_dataset_business.json"

bucket_meta = client.list_objects(Bucket = 'yelp-dataset-stevenhurwitt')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

files in s3 bucket:

raw/
raw/yelp_academic_dataset_business.json
raw/yelp_academic_dataset_checkin.json
raw/yelp_academic_dataset_review.json
raw/yelp_academic_dataset_tip.json
raw/yelp_academic_dataset_user.json


In [22]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    bucket = "yelp-dataset-stevenhurwitt"
    file = "raw/yelp_academic_dataset_business.json"
    response = client.get_object(Bucket = bucket, Key = file)
    
    file_content = response['Body'].read().decode('utf-8')
    json_file = json.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

## read json files

In [32]:
business_file = read_json("raw/yelp_academic_dataset_business.json")
checkin_file = read_json("raw/yelp_academic_dataset_checkin.json")
review_file = read_json("raw/yelp_academic_dataset_review.json")
tip_file = read_json("raw/yelp_academic_dataset_tip.json")
user_file = read_json("raw/yelp_academic_dataset_user.json")

print("read json files from s3.")

read json files from s3.


### review

In [34]:
pp.pprint(review_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### tip

In [35]:
pp.pprint(tip_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### user

In [36]:
pp.pprint(user_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### checkin

In [37]:
pp.pprint(checkin_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### business

In [48]:
pp.pprint(business_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


In [39]:
dynamodb = boto3.resource('dynamodb', endpoint_url="https://us-east-2.console.aws.amazon.com?arn=arn:aws:dynamodb:us-east-2134132211607:8000")
print('created dynamo resource.')

try:
    yelp_business = dynamodb.create_table(
            TableName='yelp.business',
            KeySchema=[
                {
                    'AttributeName': 'business_id',
                    'KeyType': 'HASH'  # Partition key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'name',
                    'AttributeType': 'S'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 25,
                'WriteCapacityUnits': 20
            }
        )
    print('created dynamo table.')

except:
    print("failed to create dynamo table.")

created dynamo resource.
failed to create dynamo table.


In [40]:
print(bucket)

yelp-dataset-stevenhurwitt


In [41]:
print(file)

raw/yelp_academic_dataset_business.json


In [43]:
print('json file has {} records with size of {} mb.'.format(len(business_file), sys.getsizeof(business_file)/1000000))
print('here is an example record:')
print('')
pp.pprint(business_file[0])

json file has 150346 records with size of 1.28316 mb.
here is an example record:

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


In [44]:
# df_pandas = business.toPandas()

In [45]:
# html_df = df_pandas.to_html()

In [46]:
# display(html_df)

In [47]:
# ! gcloud auth login --no-launch-browser

In [12]:
try:
    response = table.query(
        ProjectionExpression="user, created_at",
        ExpressionAttributeNames={"user": "created_at"},
        KeyConditionExpression=(
            Key('user').eq("ihearttyou2") &
            Key('created_at').between('2011-01-01-01T00:00:00Z', '2024-05-25T23:59:59Z')))
            
except ClientError as err:
    if err.response['Error']['Code'] == "ValidationException":
        logger.warning(
            "There's a validation error. Here's the message: %s: %s",
            err.response['Error']['Code'], err.response['Error']['Message'])
    else:
        logger.error(
            "Couldn't query for movies. Here's why: %s: %s",
            err.response['Error']['Code'], err.response['Error']['Message'])
        raise
else:
    response_items = response['Items']
    response_item_count = response.item_count
    logger.info(response_items.shape)

NameError: name 'ClientError' is not defined

In [13]:
pandas_df = response.toPandas()
html_df = pandas_df.to_html()
# display(html_df)

NameError: name 'response' is not defined

In [14]:
display(html_df)

NameError: name 'html_df' is not defined